Empecemos escogiendo nuestro generador uniforme en este caso tomaremos Mersenne Twister por su fiabilidad y por su periodo tan grande, que está implementado en la libreria numpy

In [1]:
import numpy as np

# Crear generador basado en Mersenne Twister
mt = np.random.MT19937(seed=43)
rng = np.random.Generator(mt)

print([rng.random() for _ in range(10)])

[0.6057220977812905, 0.9245321580950842, 0.6383927532488509, 0.11361476091492895, 0.7447303871101164, 0.42490588783923344, 0.46765813308716786, 0.018731721960731584, 0.0075794920916104624, 0.7952568777367439]


Queremos obtener los puntos $x_i$ que delimitan las cajas del ziggurat, con:

$$
x_i [f(x_{i-1}) - f(x_i)] = v, \quad \text{para } i = 1,\dots,N
$$

y el borde final:

$$
v = r f(r) + \int_r^{\infty} f(x)\,dx
$$

Cada caja tiene área $v$, y los $x_i$ decrecen hasta que $x_0 = 0$.
La última caja ($i = N$) termina en $x_N = r$.

Para resolverlo, necesitamos:

1. Fijar $r$ y $v$.
2. Partir de $x_N = r$.
3. Calcular recursivamente los $x_i$ hacia abajo:
   $$
   x_i = f^{-1}\left(f(x_{i+1}) + \frac{v}{x_{i+1}}\right)
   $$
   donde $f^{-1}(y)$ se obtiene como:
   $$
   f^{-1}(y) = \sqrt{-2 \ln(y)}
   $$
   (porque $f(x) = e^{-x^2/2}$).


In [2]:
n_capas=128
r=3.442619855899
v=9.91256303526217e-3

In [3]:
import numpy as np

def generar_capas(n_capas=128, r=3.442619855899, v=9.91256303526217e-3):
    """
    Calcula los bordes x_i de las cajas del ziggurat para la normal estándar (lado positivo).

    np.ndarray
        Array con los bordes x_i (de longitud n_layers), desde x_0=0 hasta x_N=r.
    """
    # Función densidad sin normalizar (lado positivo)
    f = lambda x: np.exp(-0.5 * x * x)

    # Array de bordes (x_0, ..., x_N)
    x_list = np.zeros(n_capas, dtype=np.float64)
    x_list[-1] = r  # último borde

    # Recorremos hacia abajo desde la última capa hasta la primera
    for i in range(n_capas - 2, -1, -1): # recorremos por indice de la penultima caja hacia atras
        #f_inv(y) = sqrt(-2 ln y)
        y = f(x_list[i + 1]) + v / x_list[i + 1] # calculamos el y de la caja
        x_list[i] = np.sqrt(-2.0 * np.log(y)) # obtenemos la x aplicando f_inv(y)

    return x_list


In [4]:
generar_capas()

array([9.33425437e-06, 2.72320865e-01, 3.62871431e-01, 4.26547986e-01,
       4.77437837e-01, 5.20656039e-01, 5.58692178e-01, 5.92962942e-01,
       6.24358597e-01, 6.53478639e-01, 6.80747919e-01, 7.06479611e-01,
       7.30911911e-01, 7.54230664e-01, 7.76583988e-01, 7.98092061e-01,
       8.18853907e-01, 8.38952214e-01, 8.58456843e-01, 8.77427429e-01,
       8.95915353e-01, 9.13965251e-01, 9.31616197e-01, 9.48902626e-01,
       9.65855079e-01, 9.82500804e-01, 9.98864233e-01, 1.01496740e+00,
       1.03083024e+00, 1.04647090e+00, 1.06190596e+00, 1.07715062e+00,
       1.09221888e+00, 1.10712365e+00, 1.12187692e+00, 1.13648985e+00,
       1.15097284e+00, 1.16533564e+00, 1.17958738e+00, 1.19373671e+00,
       1.20779175e+00, 1.22176023e+00, 1.23564948e+00, 1.24946650e+00,
       1.26321796e+00, 1.27691027e+00, 1.29054959e+00, 1.30414185e+00,
       1.31769278e+00, 1.33120795e+00, 1.34469275e+00, 1.35815245e+00,
       1.37159220e+00, 1.38501704e+00, 1.39843191e+00, 1.41184171e+00,
      

## Explicación paso a paso

### 1. **Qué representan las cajas**
Cada caja del ziggurat tiene **área constante = $v$**.  
Para la normal estándar, esas cajas recubren la curva $f(x) = e^{-x^2/2}$ en el lado positivo.  
La última caja termina en $x_N = r$ y la primera en $x_0 \approx 0$.

---

### 2. **Ecuación de área por caja**
El área del rectángulo $i$ es:
$$
A_i = x_i \,\big[f(x_{i-1}) - f(x_i)\big] = v
$$
De aquí se obtiene la relación:
$$
f(x_{i-1}) = f(x_i) + \frac{v}{x_i}
$$

---

### 3. **Inversión de $f$ para recuperar $x$**
Como $f(x) = e^{-x^2/2}$, su inversa en el lado positivo es:
$$
f^{-1}(y) = \sqrt{-2\ln(y)}
$$
Sustituyendo en la relación anterior:
$$
x_{i-1} = \sqrt{-2\,\ln\!\left(f(x_i) + \frac{v}{x_i}\right)}
$$
Esto permite calcular recursivamente $x_{i-1}$ conociendo $x_i$.

---

### 4. **Condición de cierre en el borde derecho**
El borde final $r$ y el área común $v$ satisfacen:
$$
v = r\,f(r) + \int_{r}^{\infty} f(x)\,dx
$$
Con los valores del paper (para 128 capas en el lado positivo):  
$$
r = 3.442619855899,\quad v = 9.91256303526217\times10^{-3}
$$
se garantiza que la base (rectángulo + cola) tiene también área $v$.

---

### 5. **Algoritmo de construcción de bordes**
1) Fijar $x_N = r$.  
2) Para $i = N-1, N-2, \dots, 1, 0$:
$$
x_i \leftarrow \sqrt{-2\,\ln\!\left(f(x_{i+1}) + \frac{v}{x_{i+1}}\right)}
$$
3) Al finalizar, se obtiene la secuencia $x_0, x_1, \dots, x_N$ con $x_0 \approx 0$ y $x_N = r$.

---

### 6. **Comprobación rápida**
La última caja debe cumplir:
$$
x_N \,\big[f(x_{N-1}) - f(x_N)\big] \approx v
$$
Numéricamente, al evaluar con doble precisión, debe dar un valor muy cercano a $9.912563\times10^{-3}$.

---

### 7. **Uso posterior**
Con los bordes $x_i$ calculados:
- Se tabulan $f_i = f(x_i)$ para todas las capas.
- Se construirán después las tablas de aceleración $k_i$ y $w_i$:
  $$
  k_i \approx \Big\lfloor 2^{31}\,\frac{x_{i-1}}{x_i}\Big\rfloor,\qquad
  w_i = \frac{x_i}{2^{31}}
  $$
Estas tablas permiten el **test rápido** del Ziggurat: con un entero con signo de 32 bits $hz$,  
se usa el índice $i = (hz \,\&\, 127)$ y se acepta si $\lvert hz\rvert < k_i$, devolviendo $x = hz \cdot w_i$.


# 🧮 Tablas precalculadas del Ziggurat (normal estándar, lado positivo)

A partir de los bordes $x_i$ (cajas de área $v$ calculadas en el paso anterior), construimos las **tablas del test rápido**:  
- $k_i$ (umbrales enteros),  
- $w_i$ (factores de escala),  
- $f_i = f(x_i)$ (densidad en los bordes).

Estas tablas hacen posible que, en $\sim 99\%$ de los casos, la muestra se obtenga con:
- **dos accesos a tabla**,  
- **una comparación** $|hz| < k_i$,  
- **una multiplicación** $x = hz \cdot w_i$.

---

## 🔢 Definiciones (consistentes con el paper)

Para la **normal estándar** usando la densidad **sin normalizar** $f(x) = e^{-x^2/2}$ y **128 capas** en el lado positivo:

- Escala entera (31 bits efectivos):  
  $$
  M = 2^{31}
  $$

- Para $i \ge 1$:
  $$
  k_i = \left\lfloor M \cdot \frac{x_{i-1}}{x_i} \right\rfloor,
  \qquad
  w_i = \frac{x_i}{M},
  \qquad
  f_i = f(x_i) = e^{-x_i^2/2}
  $$

- Fila especial $i=0$ (base + cola):
  Sea $r = x_{N}$ y $v$ el área común por capa, entonces
  $$
  q = \frac{v}{f(r)},\qquad
  k_0 = \left\lfloor \frac{r}{q} \cdot M \right\rfloor,\qquad
  w_0 = \frac{q}{M},\qquad
  f_0 = f(0) = 1
  $$

> **Intuición:** $k_i$ codifica la **proporción de anchos** $x_{i-1}/x_i$ a la escala entera $M$.  
> $w_i$ permite transformar el entero con signo $hz \in [-2^{31},2^{31}\!-\!1]$ a un real $x \approx \pm x_i$.  
> $f_i$ evita evaluar exponenciales salvo en la **ruta lenta** ($<1\%$ de los casos).


In [5]:
import numpy as np


def generar_tablas_precalculadas(x_list: np.ndarray, v: float):
    """
    Construye las tablas k, w, f del Ziggurat (normal estándar, lado positivo)
    a partir de los bordes x_i (longitud N, con x[0]≈0 y x[-1]=r) y el área común v.
    """
    x_list = np.asarray(x_list, dtype=np.float64)
    N = x_list.size
    
    if N < 2:
        raise ValueError("Se requieren al menos 2 bordes: x_0≈0 y x_{N-1}=r.")

    # Densidad sin normalizar: f(x) = exp(-x^2/2)
    f_list = np.exp(-0.5 * x_list * x_list)

    # Escala entera M = 2^31 (como en Marsaglia & Tsang para la normal)
    M = float(2**31)

    # Tablas a rellenar
    k_list = np.zeros(N, dtype=np.uint32)
    w_list = np.zeros(N, dtype=np.float64)

    # (A) Filas i>=1: relaciones directas entre bordes adyacentes
    # k[i] = floor( M * x[i-1] / x[i] )
    # w[i] = x[i] / M
    # Nota: x[0]≈0 -> k[1]=0 como en el código clásico.
    w_list[1:] = x_list[1:] / M
    with np.errstate(divide="ignore", invalid="ignore"):
        ratio = x_list[:-1] / x_list[1:]
        ratio = np.where(x_list[1:] > 0, ratio, 0.0)
    k_list[1:] = np.floor(M * ratio).astype(np.uint32)

    # (B) Fila i=0 (base + cola): usa q = v / f(r)
    r = x_list[-1]
    fr = f_list[-1]
    if fr <= 0.0:
        raise ValueError("f(r) es 0")
    q = v / fr
    # k[0] = floor( (r/q) * M ), w[0] = q / M
    k_list[0] = np.uint32(np.floor((r / q) * M))
    w_list[0] = q / M

    return k_list, w_list, f_list

## Explicación paso a paso (tablas precalculadas `k`, `w`, `f`)

### 1. **Objetivo de las tablas**
Las tablas `k`, `w` y `f` permiten que, en ≈99% de los casos, una muestra se acepte con:
- **un índice de capa** $i$,
- **una comparación entera** $|hz|<k_i$,
- **una multiplicación** $x = hz\cdot w_i$,

sin calcular exponenciales. Trabajamos en el **lado positivo** de la normal estándar con la densidad **no normalizada**:
$$
f(x)=e^{-x^2/2},\qquad x\ge 0,
$$
y con los bordes $\{\xi_i\}_{i=0}^{N}$ de las cajas de **área constante** $v$:
$$
\xi_i\,[f(\xi_{i-1})-f(\xi_i)] = v,\quad i=1,\dots,N,\qquad \xi_N=r.
$$

---

### 2. **Tabla $f$ (alturas en los bordes)**
Definimos la tabla de alturas de la densidad en los bordes:
$$
f_i \equiv f(\xi_i) = e^{-\xi_i^2/2},\qquad i=0,\dots,N.
$$
**¿Por qué?** En la **ruta lenta** (cuando falla el test rápido), la aceptación se decide con:
$$
f_i + U\,(f_{i-1}-f_i) \;<\; f(x), \qquad U\sim\mathcal U(0,1).
$$
Tabular $f_i$ evita recalcular exponenciales en los bordes; solo se evalúa $f(x)$ una vez.

---

### 3. **Escala entera y tabla $w$ (factor de escala)**
El Ziggurat usa un entero con signo de 32 bits $hz$ con **31 bits efectivos** para la magnitud. Fijamos:
$$
M = 2^{31}.
$$
Para $i\ge 1$:
$$
w_i \;=\; \frac{\xi_i}{M}.
$$
**Intuición:** si $|hz|<M$, entonces $x=hz\cdot w_i \in [-\xi_i,\,+\xi_i]$ (aplicando el signo de $hz$ para cubrir el lado negativo).

---

### 4. **Tabla $k$ (umbrales del test rápido) para $i\ge 1$**
Para $i\ge 1$:
$$
k_i \;=\; \left\lfloor\, M\cdot \frac{\xi_{i-1}}{\xi_i}\,\right\rfloor.
$$
**Papel de $k_i$:** implementa el **test rápido**:
$$
\text{si } |hz|<k_i \;\Longrightarrow\; \text{aceptar}\;\; x = hz\cdot w_i.
$$
Geométricamente, garantiza que el punto candidato cae **dentro del rectángulo** de la caja $i$ sin mirar la curva.  
Como $\xi_0\approx 0$, se cumple $k_1=\lfloor M\,\xi_0/\xi_1\rfloor=0$ (coincide con el código clásico).

---

### 5. **Caja base ($i=0$): por qué es distinta y por qué aparece $q$**
La fila $i=0$ (la **base**) es especial porque linda con la **cola** ($x>r$).  
Elegimos la base como un rectángulo de **anchura** $q$ y **altura** $f(r)$ con **área igual** a la de cualquier caja:
$$
q\,f(r)=v \;\;\Longrightarrow\;\; q=\frac{v}{f(r)}.
$$
Así, la probabilidad de caer en la base (rectángulo) coincide con la de cualquier otra caja.

Con este $q$ definimos:
$$
w_0=\frac{q}{M},\qquad
k_0=\left\lfloor \frac{r}{q}\,M\right\rfloor,\qquad
f_0=f(0)=1.
$$
**Por qué $\frac{r}{q}$ en $k_0$:** para la base no hay cociente $\xi_{-1}/\xi_0$; el **análogo geométrico** que fija el umbral entero es la razón entre el **límite derecho** $r$ y la **anchura** $q$ del rectángulo base, jugando el mismo papel que $\xi_{i-1}/\xi_i$ en las demás capas.

---

### 6. **Construcción algorítmica (de bordes a tablas)**
Dados los bordes $\{\xi_i\}$ (con $\xi_N=r$) y el $v$ del paper:
1) **Alturas**:
$$
f_i = e^{-\xi_i^2/2},\quad i=0,\dots,N.
$$
2) **Escalas** ($i\ge 1$):
$$
w_i = \frac{\xi_i}{2^{31}}.
$$
3) **Umbrales** ($i\ge 1$):
$$
k_i = \left\lfloor 2^{31}\cdot \frac{\xi_{i-1}}{\xi_i}\right\rfloor.
$$
4) **Base** ($i=0$): con $q=\dfrac{v}{f(r)}$,
$$
w_0=\frac{q}{2^{31}},\qquad
k_0=\left\lfloor \frac{r}{q}\,2^{31}\right\rfloor,\qquad
f_0=1.
$$

---

### 7. **Comprobaciones rápidas**
- **Escala correcta en la última capa**:
$$
w_N\cdot 2^{31}\;\approx\; \xi_N \;=\; r.
$$
- **Primer umbral nulo**:
$$
k_1=\left\lfloor 2^{31}\cdot \frac{\xi_0}{\xi_1}\right\rfloor = 0 \quad (\xi_0\approx 0).
$$
- **Base de área $v$**:
$$
q\,f(r)=v\;\;\Longrightarrow\;\; q=\frac{v}{f(r)}.
$$

---

### 8. **Uso posterior en el generador**
- **Test rápido (≈99%)**: con $i=(hz\ \&\ 127)$,
$$
|hz|<k_i\;\Longrightarrow\; x=hz\cdot w_i \;\;(\text{aceptado}).
$$
- **Ruta lenta (≈1%)**: se evalúa **una** exponencial y se compara con las alturas tabuladas:
$$
f_i + U\,(f_{i-1}-f_i) \;<\; f(x).
$$
- **Cola** ($i=0$ cuando toca): se usa el algoritmo específico de cola para $x>r$.


In [9]:
import numpy as np

def cola_normal(u1: float, u2: float, r: float) -> float | None:
    """
    Paso atómico de la cola derecha de N(0,1).
    Devuelve r + (-ln u1)/r si se acepta; None si se rechaza.
    """
    # Comprobación básica de rango
    if not (0.0 < u1 < 1.0 and 0.0 < u2 < 1.0):
        raise ValueError("u1 y u2 deben estar en (0,1).")

    x = -np.log(u1) / r
    y = -np.log(u2)

    # Test de aceptación de Marsaglia (1963): aceptar si 2*y > x^2
    accepted = (2.0 * y) > (x * x)
    if accepted:
        return r + x
    else:
        return None



In [40]:
cola_normal(rng.random(), rng.random(), r=r)

np.float64(3.8418021618931952)

## Explicación paso a paso (tratamiento de la cola en el Ziggurat normal)

### 1. **Qué problema resolvemos**
En la distribución normal estándar, la densidad decrece rápidamente:
$$
f(x) = e^{-x^2/2}.
$$
Para $x > r$ (por ejemplo $r \approx 3.4426$ en el Ziggurat de 128 capas), el valor de $f(x)$ es tan pequeño que  
las áreas de esas regiones son ínfimas y la probabilidad de obtener una muestra allí es muy baja.  
Sin embargo, **no podemos ignorar esa región**: el área total bajo la curva debe seguir siendo 1.

Por eso se crea un método especializado para muestrear correctamente en la **cola derecha**:
$$
X \sim \mathcal N(0,1)\ \big|\ X > r.
$$
Gracias a la simetría de la normal, la cola izquierda se obtiene simplemente aplicando un signo negativo.

---

### 2. **Fundamento del método de Marsaglia (1963)**
Marsaglia propuso un método de aceptación-rechazo muy eficiente para muestrear la cola de la normal:

1. Genera dos uniformes $U_1, U_2 \sim \mathcal U(0,1)$.
2. Calcula:
   $$
   X^\star = r + \frac{-\ln U_1}{r}, \qquad Y = -\ln U_2.
   $$
3. **Acepta** $X^\star$ si:
   $$
   2Y > \left(\frac{-\ln U_1}{r}\right)^2.
   $$
4. Si no se cumple, repite con nuevos $U_1, U_2$.

El valor final $X^\star$ tendrá densidad proporcional a $f(x) = e^{-x^2/2}$ en $x>r$.

---

### 3. **Por qué funciona**
El método transforma la cola $x>r$ en una región en el plano $(U_1,U_2)$ en la que es fácil aplicar un
**criterio de aceptación geométrico**.  
El rechazo se produce con baja probabilidad (≈ 10–12%) cuando $r\approx 3$–$3.5$,  
por lo que la eficiencia es muy alta.

Geométricamente, la condición $2Y > (\ln U_1 / r)^2$ asegura que el punto simulado cae bajo la curva de la densidad truncada.

---

### 4. **Integración con el Ziggurat**
En el Ziggurat, la **última caja (i = 0)** está formada por:

- Un **rectángulo** de altura $f(r)$ y anchura $q$,  
- Una **cola** que comienza en $r$ y continúa hasta $\infty$.

El área del rectángulo se define igual que las demás cajas:
$$
q\,f(r) = v \quad \Longrightarrow \quad q = \frac{v}{f(r)}.
$$
Por tanto, la probabilidad de caer en la “base” o en la “cola” es coherente con el resto del diseño del ziggurat.

Cuando el índice de caja es $i=0$, el algoritmo hace:
1. Genera un punto en la base rectangular con probabilidad proporcional a $r\,f(r) / v$.
2. Si cae fuera del rectángulo, entra en la **cola** y se genera una muestra con el método anterior.

El signo del número base ($hz$) se utiliza para reflejar simétricamente la muestra:  
si $hz<0$, el resultado se multiplica por $-1$.


In [7]:
#generar_capas()
k, w, f = generar_tablas_precalculadas(generar_capas(), v)

In [8]:
f

array([1.        , 0.96359969, 0.93628268, 0.91304365, 0.89228165,
       0.87324305, 0.85550061, 0.83878361, 0.82290721, 0.80773829,
       0.79317701, 0.77914609, 0.76558417, 0.75244156, 0.73967724,
       0.72725692, 0.71515151, 0.7033361 , 0.69178914, 0.68049184,
       0.66942767, 0.658582  , 0.64794182, 0.63749548, 0.62723249,
       0.61714337, 0.60721954, 0.59745315, 0.58783705, 0.57836468,
       0.56902999, 0.55982741, 0.55075179, 0.54179836, 0.53296266,
       0.52424057, 0.51562824, 0.50712205, 0.49871864, 0.49041483,
       0.48220765, 0.4740943 , 0.46607215, 0.45813872, 0.45029164,
       0.44252872, 0.43484783, 0.427247  , 0.41972433, 0.41227804,
       0.40490642, 0.39760786, 0.39038081, 0.38322381, 0.37613547,
       0.36911445, 0.3621595 , 0.35526938, 0.34844297, 0.34167914,
       0.33497685, 0.3283351 , 0.32175292, 0.31522939, 0.30876364,
       0.30235483, 0.29600216, 0.28970486, 0.28346221, 0.2772735 ,
       0.27113808, 0.2650553 , 0.25902457, 0.2530453 , 0.24711

In [271]:
hz = rng.integers(np.iinfo(np.int32).min, np.iinfo(np.int32).max, dtype=np.int32)
print("numero original:         ",hz)
print("numero original binario: ",np.binary_repr(hz))
print("longitud numero binario: ",len(np.binary_repr(hz)))
print("mascara:                 ", np.binary_repr(np.int32(127)))
print("7 ultimos bits:          ",np.binary_repr(hz)[-7:])
print("resultado AND:           ",np.binary_repr(int(abs(hz) & np.int32(127))))
print("resultado AND COMPLETO:  ",np.binary_repr(hz & np.int32(127)))
print("resultado AND COMPLETO:  ",hz & np.int32(127))

print("resultado AND NEG:       ",np.binary_repr(int(hz & np.int32(128))))
print("resultado TODO AND NEG:  ",np.binary_repr(hz & np.int32(128)))

numero original:          -599675483
numero original binario:  -100011101111100101001001011011
longitud numero binario:  31
mascara:                  1111111
7 ultimos bits:           1011011
resultado AND:            1011011
resultado AND COMPLETO:   100101
resultado AND COMPLETO:   37
resultado AND NEG:        10000000
resultado TODO AND NEG:   10000000


In [186]:
128 & 127

0

61


In [111]:
len(np.binary_repr(rng.integers(np.iinfo(np.int32).min, np.iinfo(np.int32).max, dtype=np.int32)))

29

In [107]:
np.binary_repr(hz)

'1101010011011001101011111100111'

In [108]:
len("1101010011011001101011111100111")

31

In [ ]:
import numpy as np

def sample_tail_once(rng: np.random.Generator, r: float) -> float:
    while True:
        u1 = rng.random()
        u2 = rng.random()
        x = cola_normal(u1, u2, r)
        if x is not None:
            return x  # > r


def ziggurat_norm_one(
    rng: np.random.Generator,
    k: np.ndarray,        # uint32, shape (N,)
    w: np.ndarray,        # float64, shape (N,)
    f: np.ndarray,        # float64, shape (N,), f[i]=exp(-xi[i]^2/2)
    xi: np.ndarray,       # float64, shape (N,), bordes lado positivo (xi[0]≈0, xi[-1]=r)
    r: float              # límite derecho (xi[-1])
) -> float:
    """
    Genera UNA muestra ~ N(0,1) usando el método Ziggurat (Marsaglia & Tsang, 2000),
    con tablas precalculadas (k, w, f, xi). Devuelve un float.

    Reglas:
      - Test rápido: |hz| < k[i]  -> x = hz * w[i] (aceptado)
      - Si i == 0 y falla: cola (x>r) con Marsaglia 1963; aplicar signo de hz
      - Si i > 0 y falla: tiras (rechazo local) con alturas tabuladas f[i-1], f[i]
    """
    N = k.size
    # 1) Entero base int32 (31 bits magnitud + 1 bit de signo)
    hz = rng.integers(np.iinfo(np.int32).min, np.iinfo(np.int32).max, dtype=np.int32)

    # 2) Índice de capa (asume N potencia de 2: típico N=128); si no lo es, usar % N
    iz = int(hz & (N - 1)) if (N & (N - 1)) == 0 else int(abs(hz) % N)

    # 3) Test rápido
    abs_hz = np.uint32(abs(int(hz)))
    if abs_hz < k[iz]:
        return float(hz) * float(w[iz])

    # 4) Ruta lenta
    sign = 1.0 if hz >= 0 else -1.0

    if iz == 0:
        # 4a) COLA (i == 0): reintentar hasta aceptar un valor > r y aplicar signo
        while True:
            u1 = rng.random()
            u2 = rng.random()
            x_pos = cola_normal_step_np(u1, u2, r)  # None si rechaza
            if x_pos is not None:
                return sign * x_pos
    else:
        # 4b) TIRAS (i > 0): propuesta uniforme en la base de la capa i y rechazo local
        i = iz
        while True:
            # Propuesta: x = ± U * xi[i], con U ~ Unif(0,1)
            Ux = rng.random()
            x = sign * (Ux * float(xi[i]))

            # Altura uniforme en la tira: f[i] + U2*(f[i-1] - f[i])
            Uy = rng.random()
            lhs = float(f[i]) + Uy * (float(f[i - 1]) - float(f[i]))
            rhs = float(np.exp(-0.5 * x * x))  # f(x)

            if lhs < rhs:
                return x


# ======================
# Ejemplo de ejecución:
# ======================
if __name__ == "__main__":
    # Usa lo que ya definiste antes en tu notebook:
    # rng, n_capas, r, v, generar_capas, generar_tablas_precalculadas, cola_normal_step_np
    x_list = generar_capas(n_capas, r, v)
    k, w, f = generar_tablas_precalculadas(x_list, v)
    muestras = ziggurat_norm(500_000, rng, x_list, k, w, f, r)

    print("Media   ≈ 0:", float(np.mean(muestras)))
    print("DesvTip ≈ 1:", float(np.std(muestras)))
    # Percentiles de control
    print("p99.5:", float(np.percentile(muestras, 99.5)))
    print("p99.9:", float(np.percentile(muestras, 99.9)))
